## Job description from google jobs

In [1]:
# import pandas as pd
# # from serpapi import GoogleSearch
# import sqlite3
# import datetime as dt
# import http.client
# import json
# import config
# import urllib.parse
# import os
# from sqlalchemy import create_engine
# import psycopg2

# def google_job_search(job_title, city_state):
#     '''
#     job_title(str): "Data Scientist", "Data Analyst"
#     city_state(str): "Denver, CO"
#     post_age,(str)(optional): "3day", "week", "month"
#     '''
#     query = f"{job_title} {city_state}"
#     params = {
#         "engine": "google_jobs",
#         "q": query,
#         "hl": "en",
#         "api_key": os.getenv('SerpAPIkey'),
#         # "chips": f"date_posted:{post_age}",
#     }

#     query_string = urllib.parse.urlencode(params, quote_via=urllib.parse.quote)

#     conn = http.client.HTTPSConnection("serpapi.webscrapingapi.com/v1")
#     try:
#         conn.request("GET", f"/v1?{query_string}")
#         res = conn.getresponse()
#         try:
#             data = res.read()
#         finally:
#             res.close()
#     finally:
#         conn.close()

#     try:
#         json_data = json.loads(data.decode("utf-8"))
#         jobs_results = json_data['google_jobs_results']
#         job_columns = ['title', 'company_name', 'location', 'description']
#         df = pd.DataFrame(jobs_results, columns=job_columns)
#         return df
#     except (KeyError, json.JSONDecodeError) as e:
#         print(f"Error occurred for search: {job_title} in {city_state}")
#         print(f"Error message: {str(e)}")
#         return None

# def sql_dump(df, table):
#     engine = create_engine(f"postgresql://{os.getenv('MasterName')}:{os.getenv('MasterPass')}@{os.getenv('RDS_EndPoint')}:5432/postgres")
#     with engine.connect() as conn:
#         df.to_sql(table, conn, if_exists='append', chunksize=1000, method='multi', index=False)

# def main(job_list, city_state_list):
#     for job in job_list:
#         for city_state in city_state_list:
#             df_10jobs = google_job_search(job, city_state)
#             if df_10jobs is not None:
#                 print(f'City: {city_state} Job: {job}')
#                 print(df_10jobs.shape)
#                 date = dt.datetime.today().strftime('%Y-%m-%d')
#                 df_10jobs['retrieve_date'] = date
#                 sql_dump(df_10jobs, 'datajobs24')

#     return None

In [2]:
# job_list= ["Machine Learning Engineer", "Data Scientist", "Generative AI Engineer", "Solutions Engineer", "LLM Engineer"]
# simple_city_state_list= ["Menlo Park CA", "Palo Alto CA", "San Francisco CA", "Mountain View CA"]
# sample = main(job_list, simple_city_state_list)

In [1]:
import pandas as pd
import sqlite3
import datetime as dt
import http.client
import json
import urllib.parse
import os
from sqlalchemy import create_engine
from concurrent.futures import ThreadPoolExecutor, as_completed

def google_job_search(job_title, city_state, start=0):
    '''
    job_title(str): "Data Scientist", "Data Analyst"
    city_state(str): "Denver, CO"
    post_age,(str)(optional): "3day", "week", "month"
    '''
    query = f"{job_title} {city_state}"
    params = {
        "api_key": os.getenv('SerpAPIkey'),
        "engine": "google_jobs",
        "q": query,
        "hl": "en",
        "start": start,
        # "chips": f"date_posted:{post_age}",
    }

    query_string = urllib.parse.urlencode(params, quote_via=urllib.parse.quote)

    conn = http.client.HTTPSConnection("serpapi.webscrapingapi.com")
    try:
        conn.request("GET", f"/v1?{query_string}")
        res = conn.getresponse()
        try:
            data = res.read()
        finally:
            res.close()
    finally:
        conn.close()

    try:
        json_data = json.loads(data.decode("utf-8"))
        jobs_results = json_data['google_jobs_results']
        job_columns = ['title', 'company_name', 'location', 'description', 'extensions', 'job_id']
        df = pd.DataFrame(jobs_results, columns=job_columns)
        return df
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error occurred for search: {job_title} in {city_state}")
        print(f"Error message: {str(e)}")
        return None

def sql_dump(df, table):
    engine = create_engine(f"postgresql://{os.getenv('MasterName')}:{os.getenv('MasterPass')}@{os.getenv('RDS_EndPoint')}:5432/postgres")
    with engine.connect() as conn:
        df.to_sql(table, conn, if_exists='append', chunksize=20, method='multi', index=False)
        print(f"Dumped {df.shape} to SQL table {table}")

def process_batch(job, city_state, start):
    df_10jobs = google_job_search(job, city_state, start)
    if df_10jobs is not None:
        print(f'City: {city_state} Job: {job} Start: {start}')
        print(df_10jobs.shape)
        date = dt.datetime.today().strftime('%Y-%m-%d')
        df_10jobs['retrieve_date'] = date
        df_10jobs.drop_duplicates(subset=['job_id', 'company_name'], inplace=True)
        rows_affected = sql_dump(df_10jobs, 'usajobs24')
        print(f"Rows affected: {rows_affected}")

def main(job_list, city_state_list):
    with ThreadPoolExecutor() as executor:
        futures = []
        for job in job_list:
            for city_state in city_state_list:
                for start in range(0, 2):
                    future = executor.submit(process_batch, job, city_state, start)
                    futures.append(future)

        for future in as_completed(futures):
            future.result()

In [2]:
job_list = ["Data Analyst", "Data Engineer", "Big Data Engineer"]
simple_city_state_list = ["Menlo Park CA", "Palo Alto CA", "San Francisco CA", "Mountain View CA"]
main(job_list, simple_city_state_list)

City: Menlo Park CA Job: Data Engineer Start: 1
(10, 6)
City: San Francisco CA Job: Data Engineer Start: 0
(9, 6)
City: Mountain View CA Job: Data Analyst Start: 0
(10, 6)
City: Menlo Park CA Job: Data Analyst Start: 1
(10, 6)
City: Palo Alto CA Job: Data Analyst Start: 1
(10, 6)
City: San Francisco CA Job: Data Analyst Start: 0
(10, 6)
City: Menlo Park CA Job: Data Engineer Start: 0
(10, 6)
City: Palo Alto CA Job: Data Analyst Start: 0
(10, 6)
Dumped (10, 7) to SQL table leoTestTable
Rows affected: None
Dumped (10, 7) to SQL table leoTestTable
Rows affected: None
Dumped (10, 7) to SQL table leoTestTableDumped (9, 7) to SQL table leoTestTable
Rows affected: None

Rows affected: None
Dumped (10, 7) to SQL table leoTestTable
Rows affected: None
Dumped (10, 7) to SQL table leoTestTable
Rows affected: None
Dumped (10, 7) to SQL table leoTestTable
Rows affected: None
Dumped (10, 7) to SQL table leoTestTable
Rows affected: None
City: Mountain View CA Job: Data Engineer Start: 1
(10, 6)
Dumpe

### Great now that we have written some data lets read it.

In [31]:
import pandas as pd
from sqlalchemy import create_engine

def read_data_from_db(table_name):
    engine = create_engine(f"postgresql://{os.getenv('MasterName')}:{os.getenv('MasterPass')}@{os.getenv('RDS_EndPoint')}:5432/postgres")
    
    try:
        with engine.connect() as conn:
            query = f'SELECT * FROM "{table_name}"'
            df = pd.read_sql(query, conn)
            return df
    except Exception as e:
        print(f"Error occurred while reading data from the database: {str(e)}")
        return None

data24_df = read_data_from_db('usajobstest')

In [32]:
data24_df.shape

(417, 7)

In [33]:
data24_df

,title,company_name,location,description,extensions,job_id,retrieve_date
0,Business Intelligence Analyst,Nuvolum,"San Francisco, CA","Nuvolum combines innovative, data-driven strat...","{""3 days ago"",Full-time,""No degree mentioned""}",eyJqb2JfdGl0bGUiOiJCdXNpbmVzcyBJbnRlbGxpZ2VuY2...,2024-05-04
1,Sr. Strategy and Business Intelligence Analyst,Sunrun,"San Francisco, CA (+1 other)",Everything we do at Sunrun is driven by a dete...,"{""12 days ago"",Full-time,""Health insurance"",""D...",eyJqb2JfdGl0bGUiOiJTci4gU3RyYXRlZ3kgYW5kIEJ1c2...,2024-05-04
2,Business Intelligence Analyst,Side,Anywhere,"Side, Inc. seeks Business Intelligence Analyst...","{""11 days ago"",""151,736–157,000 a year"",""Work ...",eyJqb2JfdGl0bGUiOiJCdXNpbmVzcyBJbnRlbGxpZ2VuY2...,2024-05-04
3,Senior Business Intelligence Developer,TekNavigators Staffing,"San Francisco, CA",Role: Senior BI Developer\n\nLocation: San Fra...,"{""20 hours ago"",Contractor,""No degree mentioned""}",eyJqb2JfdGl0bGUiOiJTZW5pb3IgQnVzaW5lc3MgSW50ZW...,2024-05-04
4,Senior Business Intelligence Analyst,FIS Fidelity National Information Services,"San Francisco, CA",Position Type : Full time Type Of Hire : Exper...,"{""19 days ago"",Full-time}",eyJqb2JfdGl0bGUiOiJTZW5pb3IgQnVzaW5lc3MgSW50ZW...,2024-05-04
...,...,...,...,...,...,...,...
412,Business Intelligence Analyst - Diabetes Marke...,Medtronic,Anywhere,Careers that Change Lives\n\nWe are looking fo...,"{""10 days ago"",""Work from home"",Full-time,""No ...",eyJqb2JfdGl0bGUiOiJCdXNpbmVzcyBJbnRlbGxpZ2VuY2...,2024-05-04
413,"IT Analyst, Business Intelligence/Data Warehou...",Keck Medicine of USC,"Alhambra, CA",Actively design and develop ETL solutions that...,"{""13 days ago"",Full-time}",eyJqb2JfdGl0bGUiOiJJVCBBbmFseXN0LCBCdXNpbmVzcy...,2024-05-04
414,"Director, Business Intelligence",Deutsch LA,"Los Angeles, CA","DIRECTOR, BUSINESS INTELLIGENCE\n\nWe are seek...","{""3 days ago"",Full-time,""No degree mentioned""}",eyJqb2JfdGl0bGUiOiJEaXJlY3RvciwgQnVzaW5lc3MgSW...,2024-05-04
415,Business Intelligence Programmer 1,U.S. Bank,"Los Angeles, CA","At U.S. Bank, we’re on a journey to do our bes...","{""3 days ago"",Full-time,""Health insurance"",""De...",eyJqb2JfdGl0bGUiOiJCdXNpbmVzcyBJbnRlbGxpZ2VuY2...,2024-05-04


In [34]:
# get the list of unique title, company_name pairs
title_company = data24_df[['title', 'company_name', 'description']].drop_duplicates()

In [35]:
data24_df

,title,company_name,location,description,extensions,job_id,retrieve_date
0,Business Intelligence Analyst,Nuvolum,"San Francisco, CA","Nuvolum combines innovative, data-driven strat...","{""3 days ago"",Full-time,""No degree mentioned""}",eyJqb2JfdGl0bGUiOiJCdXNpbmVzcyBJbnRlbGxpZ2VuY2...,2024-05-04
1,Sr. Strategy and Business Intelligence Analyst,Sunrun,"San Francisco, CA (+1 other)",Everything we do at Sunrun is driven by a dete...,"{""12 days ago"",Full-time,""Health insurance"",""D...",eyJqb2JfdGl0bGUiOiJTci4gU3RyYXRlZ3kgYW5kIEJ1c2...,2024-05-04
2,Business Intelligence Analyst,Side,Anywhere,"Side, Inc. seeks Business Intelligence Analyst...","{""11 days ago"",""151,736–157,000 a year"",""Work ...",eyJqb2JfdGl0bGUiOiJCdXNpbmVzcyBJbnRlbGxpZ2VuY2...,2024-05-04
3,Senior Business Intelligence Developer,TekNavigators Staffing,"San Francisco, CA",Role: Senior BI Developer\n\nLocation: San Fra...,"{""20 hours ago"",Contractor,""No degree mentioned""}",eyJqb2JfdGl0bGUiOiJTZW5pb3IgQnVzaW5lc3MgSW50ZW...,2024-05-04
4,Senior Business Intelligence Analyst,FIS Fidelity National Information Services,"San Francisco, CA",Position Type : Full time Type Of Hire : Exper...,"{""19 days ago"",Full-time}",eyJqb2JfdGl0bGUiOiJTZW5pb3IgQnVzaW5lc3MgSW50ZW...,2024-05-04
...,...,...,...,...,...,...,...
412,Business Intelligence Analyst - Diabetes Marke...,Medtronic,Anywhere,Careers that Change Lives\n\nWe are looking fo...,"{""10 days ago"",""Work from home"",Full-time,""No ...",eyJqb2JfdGl0bGUiOiJCdXNpbmVzcyBJbnRlbGxpZ2VuY2...,2024-05-04
413,"IT Analyst, Business Intelligence/Data Warehou...",Keck Medicine of USC,"Alhambra, CA",Actively design and develop ETL solutions that...,"{""13 days ago"",Full-time}",eyJqb2JfdGl0bGUiOiJJVCBBbmFseXN0LCBCdXNpbmVzcy...,2024-05-04
414,"Director, Business Intelligence",Deutsch LA,"Los Angeles, CA","DIRECTOR, BUSINESS INTELLIGENCE\n\nWe are seek...","{""3 days ago"",Full-time,""No degree mentioned""}",eyJqb2JfdGl0bGUiOiJEaXJlY3RvciwgQnVzaW5lc3MgSW...,2024-05-04
415,Business Intelligence Programmer 1,U.S. Bank,"Los Angeles, CA","At U.S. Bank, we’re on a journey to do our bes...","{""3 days ago"",Full-time,""Health insurance"",""De...",eyJqb2JfdGl0bGUiOiJCdXNpbmVzcyBJbnRlbGxpZ2VuY2...,2024-05-04


In [36]:
# from typing import List, Optional
# from langchain_core.pydantic_v1 import BaseModel, Field

# class CompanyOverview(BaseModel):
#     """
#     A model for capturing key information about the company offering the job.
    
#     Extract relevant details about the company from the job description, 
#     including a brief overview of its industry and products, its mission and 
#     values, size, and location(s).
    
#     Focus on capturing the most salient points that give a well-rounded picture
#     of the company and its culture.
#     """

#     about: Optional[str] = Field(
#         None, 
#         description="""Brief description of the company, its industry, products, services, 
#                     and any notable achievements or differentiators"""
#     )

#     mission_and_values: Optional[str] = Field(
#         None,
#         description="""Company mission, vision, values, and culture, including commitments 
#                     to diversity, inclusion, social responsibility, and work-life balance"""
#     )
    
#     size: Optional[str] = Field(
#         None,
#         description="Details about company size, such as number of employees")
    
#     locations: Optional[str] = Field(
#         None,
#         description="""Geographic presence of the company, including headquarters, 
#                     offices, and any remote work options"""
#     )
    
#     city: Optional[str] = Field(None, description="City where the company is located")
    
#     state: Optional[str] = Field(None, description="State where the company is located")


# class RoleSummary(BaseModel):
#     """
#     A model for capturing the key summary points about the job role.
    
#     Extract the essential high-level details about the role from the job description,
#     such as the job title, the team or department the role belongs to, the role type, 
#     and any remote work options.
    
#     Prioritize information that helps understand the overall scope and positioning 
#     of the role within the company.
#     """
    
#     title: str = Field(..., description="Title of the job role")
    
#     team_or_department: Optional[str] = Field(
#         None,
#         description="""Team, department, or business unit the role belongs to, 
#                     including any collaborations with other teams"""
#     )
    
#     role_type: Optional[str] = Field(
#         None,
#         description="Type of role (full-time, part-time, contract, etc.)"
#     )
    
#     remote: Optional[str] = Field(
#         None,
#         description="Remote work options for the role (full, hybrid, none)"
#     )

# class ResponsibilitiesAndQualifications(BaseModel):
#     """
#     A model for capturing the key responsibilities, requirements, and preferred 
#     qualifications for the job role.

#     Extract the essential duties and expectations of the role, the mandatory 
#     educational background and experience required, and any additional skills 
#     or characteristics that are desirable but not strictly necessary.

#     The goal is to provide a clear and comprehensive picture of what the role 
#     entails and what qualifications the ideal candidate should possess.
#     """

#     responsibilities: List[str] = Field(
#         description="""The core duties, tasks, and expectations of the role, encompassing 
#                     areas such as metrics, theories, business understanding, product 
#                     direction, systems, leadership, decision making, strategy, and 
#                     collaboration, as described in the job description"""
#     )

#     required_qualifications: List[str] = Field(
#         description="""The essential educational qualifications (e.g., Doctorate, 
#                     Master's, Bachelor's degrees in specific fields) and years of 
#                     relevant professional experience that are mandatory for the role, 
#                     including any alternative acceptable combinations of education 
#                     and experience, as specified in the job description"""
#     )
    
#     preferred_qualifications: List[str] = Field(
#         description="""Any additional skills, experiences, characteristics, or domain 
#                     expertise that are valuable for the role but not absolute 
#                     requirements, such as proficiency with specific tools/technologies, 
#                     relevant soft skills, problem solving abilities, and industry 
#                     knowledge, as mentioned in the job description as preferred or 
#                     nice-to-have qualifications"""
#     )
    
# class CompensationAndBenefits(BaseModel):
#     """
#     A model for capturing the compensation and benefits package for the job role.
    
#     Extract details about the salary or pay range, bonus and equity compensation, 
#     benefits, and perks from the job description.
    
#     Aim to provide a comprehensive view of the total rewards offered for the role,
#     including both monetary compensation and non-monetary benefits and perks.
#     """
    
#     salary_or_pay_range: Optional[str] = Field(
#         None,
#         description="""The salary range or hourly pay range for the role, including 
#                     any specific numbers or bands mentioned in the job description"""
#     )
    
#     bonus_and_equity: Optional[str] = Field(
#         None,
#         description="""Any information about bonus compensation, such as signing bonuses, 
#                     annual performance bonuses, or other incentives, as well as details 
#                     about equity compensation like stock options or RSUs"""
#     )
    
#     benefits: Optional[List[str]] = Field(
#         None,
#         description="""A list of benefits offered for the role, such as health insurance, 
#                     dental and vision coverage, retirement plans (401k, pension), paid 
#                     time off (vacation, sick days, holidays), parental leave, and any 
#                     other standard benefits mentioned in the job description"""
#     )
    
#     perks: Optional[List[str]] = Field(
#         None,
#         description="""A list of additional perks and amenities offered, such as free food 
#                     or snacks, commuter benefits, wellness programs, learning and development 
#                     stipends, employee discounts, or any other unique perks the company 
#                     provides to its employees, as mentioned in the job description"""
#     )

# class JobDescription(BaseModel):
#     """Extracted information from a job description."""
#     company_overview: CompanyOverview
#     role_summary: RoleSummary
#     responsibilities_and_qualifications: ResponsibilitiesAndQualifications
#     compensation_and_benefits: CompensationAndBenefits

In [ ]:
import sys
sys.path.append('../utils')

from job_desc_pydantic import JobDescription

In [37]:
from typing import List, Optional

from langchain.chains import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()

True

In [38]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at identifying key aspects of job descriptions. Your task is to extract important information from a raw job description and organize it into a structured format using the ResponsibilitiesAndQualifications class.

                When parsing the job description, your goal is to capture as much relevant information as possible in the appropriate fields of the class. This includes:

                1. All key responsibilities and duties of the role, covering the full range of tasks and expectations.
                2. The required educational qualifications and years of experience, including different acceptable combinations.
                3. Any additional preferred skills, experiences, and characteristics that are desirable for the role.

                Avoid summarizing or paraphrasing the information. Instead, extract the details as closely as possible to how they appear in the original job description. The aim is to organize and structure the raw data, not to condense or interpret it.

                Some specific things to look out for:
                - Responsibilities related to metrics, theories, business understanding, product direction, systems, leadership, decision making, strategy, and collaboration
                - Required degrees (Doctorate, Master's, Bachelor's) in relevant fields, along with the corresponding years of experience
                - Preferred qualifications like years of coding experience, soft skills, problem solving abilities, and domain expertise

                If any of these details are missing from the job description, simply omit them from the output rather than trying to infer or fill in the gaps.

                The structured data you extract will be used for further analysis and insights downstream, so err on the side of including more information rather than less. The key is to make the unstructured job description data more organized and manageable while still retaining all the important details.
            """,
        ),
        ("human", "{text}"),
    ]
)

In [44]:
llm = ChatGroq(model_name="llama3-70b-8192")

extractor = prompt | llm.with_structured_output(
    schema=JobDescription,
    method="function_calling",
    include_raw=False,
)

/Users/leowalker/anaconda3/envs/datajobs/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The method `ChatGroq.with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [42]:
test_description = title_company['description'][2]

In [157]:
jobdesc = extractor.invoke(test_description)

In [158]:
import json

def pretty_print_pydantic(obj):
    print(json.dumps(obj.dict(), indent=4))

# Example usage
pretty_print_pydantic(jobdesc)
jobdesc

{
    "company_overview": {
        "about": "Microsoft is a leading technology company responsible for delivering the quality experience to over 500M+ monthly active users around the world in Microsoft\u2019s search engine, Bing.",
        "mission_and_values": "Empower every person and every organization on the planet to achieve more.",
        "size": "500M+ users",
        "locations": "Global",
        "city": "Redmond",
        "state": null
    },
    "role_summary": {
        "title": "Principal Data Scientist",
        "team_or_department": "Search + Distribution (S+D) team",
        "role_type": "Full-time",
        "remote": "N/A"
    },
    "responsibilities_and_qualifications": {
        "responsibilities": [
            "Define, invent, and deliver online and offline behavioral and human labeled metrics which accurately measure the satisfaction and success of our customers interacting with Search.",
            "Apply behavioral game theory and social science understandin

JobDescription(company_overview=CompanyOverview(about='Microsoft is a leading technology company responsible for delivering the quality experience to over 500M+ monthly active users around the world in Microsoft’s search engine, Bing.', mission_and_values='Empower every person and every organization on the planet to achieve more.', size='500M+ users', locations='Global', city='Redmond', state=None), role_summary=RoleSummary(title='Principal Data Scientist', team_or_department='Search + Distribution (S+D) team', role_type='Full-time', remote='N/A'), responsibilities_and_qualifications=ResponsibilitiesAndQualifications(responsibilities=['Define, invent, and deliver online and offline behavioral and human labeled metrics which accurately measure the satisfaction and success of our customers interacting with Search.', 'Apply behavioral game theory and social science understanding to get the quality work out of crowd workers from around the world', 'Develop deep understanding of business me

In [95]:
import pprint
pp = pprint.PrettyPrinter(width=80)
pp.pprint(test_description)


('The Search + Distribution (S+D) team is the leading applied artificial '
 'intelligence team at Microsoft responsible for delivering the quality '
 'experience to over 500M+ monthly active users around the world in '
 'Microsoft’s search engine, Bing. Our responsibilities include delivering '
 'competitive search results, differentiated experiences, and product and '
 'business growth. We are constantly applying the... latest state of the art '
 'AI technologies to our product and also transferring this technology to '
 'other groups across the company.\n'
 '\n'
 'We sre seeking experienced data scientist to solve cutting-edge metrics and '
 'measurement problems in the space of Search, and lead cross-team '
 'initiatives. We believe metrics play a key role in executing on the strategy '
 'for building the final product.\n'
 '\n'
 'A critical part of the role is to advance our A/B experimentation '
 'capabilities for Bing and Microsoft Copilot by introducing advanced, '
 'powerful fu